In [15]:
import os
import torch
from torch import nn as nn
from torchvision import datasets, models, transforms
from torch import optim

In [16]:
ddir = '/content/drive/MyDrive/study/study by lecture/implementation/dataset/hym_data'

batch_size = 4
num_workers = 2

data_transformers = {
    'train' : transforms.Compose(
        [
          transforms.RandomResizedCrop(224),
          transforms.RandomHorizontalFlip(),
          transforms.ToTensor(),
          transforms.Normalize([0.490, 0.449, 0.411],[0.231, 0.221, 0.230])

        ]
    ),
    'val' : transforms.Compose(
        [
          transforms.Resize(256),
          transforms.CenterCrop(224),
          transforms.ToTensor(),
          transforms.Normalize([0.490, 0.449, 0.411],[0.231, 0.221, 0.230])
        ]
    )
}

img_data = {
    k : datasets.ImageFolder(os.path.join(ddir, k), data_transformers[k]) for k in ['train', 'val']
    }

dloaders = {
    k : torch.utils.data.DataLoader(img_data[k], batch_size = batch_size, shuffle = True, num_workers = num_workers) for k in ['train','val']
}

dset_sizes = {x:len(img_data[x]) for x in ['train','val']}
classes = img_data['train'].classes

dvc = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [17]:
from torch.nn.modules.batchnorm import BatchNorm2d
import time
import torch.nn as nn

class BasicBlock(nn.Module):
  def __init__(self, in_channels, out_channels, stride = 1):
    super().__init__()
    
    self.residual_function = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1, bias = False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace = True),
        nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1, bias = False),
        nn.BatchNorm2d(out_channels)
    )

    self.shortcut = nn.Sequential()

    if stride != 1 or in_channels != out_channels:
      self.shortcut = nn.Sequential(
          nn.Conv2d(in_channels, out_channels, kernel_size=1, stride = stride, bias = False),
          nn.BatchNorm2d(out_channels)
      )

  def forward(self, x):
    return nn.ReLU(inplace=True)(self.residual_function(x) + self.shortcut(x))


class ResNet(nn.Module):
  def __init__(self, block, num_block, num_classes = 100):
    super().__init__()
    self.in_channels = 64

    self.conv1 = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size = 3, padding = 1, bias = False),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace = True))
    
    self.conv2_x = self._make_layer(block, 64, num_block[0], 1)
    self.conv3_x = self._make_layer(block, 128, num_block[1], 2)
    self.conv4_x = self._make_layer(block, 256, num_block[2], 2)
    self.conv5_x = self._make_layer(block, 512, num_block[3], 2)
    self.avg_pool = nn.AdaptiveAvgPool2d((1,1))

    self.fc = nn.Linear(512, num_classes)

  def _make_layer(self, block, out_channels, num_blocks, stride):

    strides = [stride] + [1] * (num_blocks - 1)
    layers = []
    for stride in strides:
      layers.append(block(self.in_channels, out_channels, stride))
      self.in_channels = out_channels

    return nn.Sequential(*layers)

  def forward(self, x):
    output = self.conv1(x)
    output = self.conv2_x(output)
    output = self.conv3_x(output)
    output = self.conv4_x(output)
    output = self.conv5_x(output)
    output = self.avg_pool(output)
    output = output.view(output.size(0), -1)
    output = self.fc(output)

    return output

def resnet34(num_classes = 100):
  return ResNet(BasicBlock, [3, 4, 6, 3], num_classes = num_classes) 

def resnet18(num_classes = 100):
  return ResNet(BasicBlock, [2, 2, 2, 2], num_classes = num_classes) 


In [18]:
model = resnet34(2)
if torch .cuda.is_available():
  model = model.cuda()
print(model)

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2_x): Sequential(
    (0): BasicBlock(
      (residual_function): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (residual_function): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_

In [19]:
def train(model, loss_func, optimizer, epochs=10):
   start = time.time()

   accuracy = 0.0

   for e in range(epochs):
        print(f'Epoch number {e}/{epochs - 1}')
        print('=' * 20)

        for dset in ['train', 'val']:
            if dset == 'train':
                model.train()  
            else:
                model.eval() 

            loss = 0.0
            successes = 0

            for imgs, tgts in dloaders[dset]:
                imgs = imgs.to(dvc)
                tgts = tgts.to(dvc)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(dset == 'train'):
                    ops = model(imgs)
                    _, preds = torch.max(ops, 1)
                    loss_curr = loss_func(ops, tgts)
                    
                    if dset == 'train':
                        loss_curr.backward()
                        optimizer.step()

                loss += loss_curr.item() * imgs.size(0)
                successes += torch.sum(preds == tgts.data)
          
            loss_epoch = loss / dset_sizes[dset]
            accuracy_epoch = successes.double() / dset_sizes[dset]

            print(f'{dset} loss in this epoch: {loss_epoch}, accuracy in this epoch: {accuracy_epoch}')
            if dset == 'val' and accuracy_epoch > accuracy:
                accuracy = accuracy_epoch      

   time_delta = time.time() - start
   print(f'Training finished in {time_delta // 60}mins {time_delta % 60}secs')
   print(f'Best validation set accuracy: {accuracy}')


   return model


In [20]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0001)
pretrained_model = train(model, loss_func, optimizer, epochs = 5)

Epoch number 0/4
train loss in this epoch: 0.8554288442017602, accuracy in this epoch: 0.5491803278688525
val loss in this epoch: 1.9843084956306258, accuracy in this epoch: 0.5555555555555556
Epoch number 1/4
train loss in this epoch: 0.7363552973895776, accuracy in this epoch: 0.569672131147541
val loss in this epoch: 0.9173591261595683, accuracy in this epoch: 0.6339869281045751
Epoch number 2/4
train loss in this epoch: 0.6741929459767263, accuracy in this epoch: 0.6024590163934427
val loss in this epoch: 1.3598777953705756, accuracy in this epoch: 0.6666666666666667
Epoch number 3/4
train loss in this epoch: 0.7003079100710446, accuracy in this epoch: 0.5860655737704918
val loss in this epoch: 0.6423979463920094, accuracy in this epoch: 0.7320261437908497
Epoch number 4/4
train loss in this epoch: 0.6192502137578901, accuracy in this epoch: 0.6844262295081968
val loss in this epoch: 0.6462837760062778, accuracy in this epoch: 0.6535947712418301
Training finished in 2.0mins 39.2841